## Community Quotient Graphs

In this notebook, we mainly create the panels for Figure 4 and the analogue figures in the SI.
We also compute the table describing the attribute scaling used in the visualizations.

### Channel Coding

#### General
- `edge color` reflects inter-cluster number of references (raw count, color scaled from 0 to 100)
- `node color` (if not grey) reflects cluster family
- `node size` reflects number of tokens
- `layout` is force-directed, seeded, and computed based on all edges

#### Law Name Graphs by Title (US) or Gesetzesname (DE)
- `nodes` are all titles for US, laws with `tokens >= 5000` for DE
- `labels` for US are title numbers, for DE are law abbreviations
- `size divisor` (number of tokens/size divisor = node size) is 4000 in US (as it is for US and DE in the community quotient graphs) but 100 for DE so we can see stuff

> i.e.: sizes BETWEEN countries are NOT comparable for the law name quotient graphs (but they ARE comparable for the community quotient graphs)!

#### Chapter Graphs by Chapter (where available)/Title (US) or Buch (where available)/Gesetzesname (DE)
- `nodes` with more than `5000` tokens are shown
- `labels` are `title/chapter` resp. `lawname/book`, where applicable, else `title` resp. `lawname`
(only the labels for the 50 largest nodes are drawn)

#### Quotient Graphs by Community IDs for 1994 and 2018, US vs DE
- current node drawing thresholds around the 0.9 quantile of community sizes in the US (and seems to work decently for DE, too) 
- `nodes` with `tokens >= 100000 (1994)` resp. >= `150000 (2018)` are shown

### Preparations

In [ ]:
%run de_hue_order_alignment.py

In [ ]:
import numpy as np
import seaborn as sns
import networkx as nx
from matplotlib import pyplot as plt
from quantlaw.utils.networkx import quotient_graph, sequence_graph
from legal_data_clustering.utils.graph_api import cluster_families, get_clustering_result, add_community_to_graph
from legal_data_clustering.utils.nodes_merging import quotient_graph_with_merge

In [ ]:
%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import cm
import pandas as pd

min_max_df = pd.DataFrame()

In [ ]:
def internal_references_for_community(G, cG):
    community_ids = set(nx.get_node_attributes(G, 'community').values())
    assert len(community_ids) > 0
    
    G_reference_edges = nx.MultiDiGraph()
    G_reference_edges.add_nodes_from(G.nodes(data=True))
    G_reference_edges.add_edges_from(e for e in G.edges(data=True) if e[-1]['edge_type'] == 'reference')

    internat_references = dict()

    for com in sorted(community_ids):
        com_nodes = [n for n, n_com in G.nodes(data='community') if n_com == com]
        G_sub = G_reference_edges.subgraph(com_nodes)
        internat_references[com] = len(G_sub.edges)
    nx.set_node_attributes(cG, internat_references, 'internal_references')

In [ ]:
def make_weighted(mG):
    G = nx.DiGraph()
    G.add_nodes_from(mG.nodes(data=True))
    for u,v,data in mG.edges(data=True):
        w = data['weight'] if 'weight' in data else 1.0
        if G.has_edge(u,v):
            G[u][v]['weight'] += w
        else:
            G.add_edge(u, v, weight=w)
    return G

def extract_lawname_from_nodekey(nodekey, country_code):
    """
    :param nodekey: the key of a node in a statute graph, e.g., the crossreference graph
    :param country_code: us|de
    :return: the short identifier of the law the node belongs to
    """
    if country_code.lower() == "de":
        return nodekey.split("_")[1]
    else:
        return nodekey.split("_")[0]


def make_lawname_graph_for_snapshot(snapshot, country_code, weight_threshold, size_threshold, size_divisor):
    result = get_clustering_result(f'../../legal-networks-data/{country_code.lower()}/11_cluster_results/{snapshot}_0-0_1-0_-1_a-infomap_n100_m1-0_s0_c1000.json',country_code,'seqitems', path_prefix='../')
    G = result.graph
    qG = quotient_graph(G, node_attribute='law_name', edge_types=['reference'], root_level=-1, self_loops=True)
    qG = make_weighted(qG)
    # this label is suitable only for the US case, pass the DE case manually
    nx.set_node_attributes(qG, {node_id:node_id.split('-', 1)[0].split(' ')[-1] for node_id in qG.nodes()}, name='label')
    nx.set_node_attributes(qG, {node_id:node_size/size_divisor for node_id, node_size in dict(qG.nodes(data='tokens_n', default=0)).items()}, 
                           name='node_size')
    nx.set_node_attributes(qG, {u:w for u,v,w in qG.edges(data='weight') 
                                if u == v}, name='self_references'
                          )
    nx.set_node_attributes(qG, {law_name:len([n for n, data in G.nodes(data=True) 
                                              if data['law_name'] == law_name and data['type'] == 'seqitem'])
                                for law_name in qG.nodes()
                               }, name='n_seqitems'
                          )
    nx.set_node_attributes(qG, {law_name:data['self_references']/data['tokens_n'] 
                                if data.get('self_references',0) > 0 else 0.
                                for law_name, data in qG.nodes(data=True)
                               }, name='internal_density'
                          )
    qG.remove_edges_from([(u,v) for u,v,w in qG.edges(data='weight') if w <= weight_threshold])
    qG.remove_edges_from([(u,v) for u,v in qG.edges() if u == v])
    qG = nx.subgraph(qG, [n for n, size in qG.nodes(data='tokens_n', default=0) if size >= size_threshold])
    return qG


def make_community_graph_for_snapshot(snapshot, country_code, weight_threshold, size_threshold, size_divisor):
    result = get_clustering_result(f'../../legal-networks-data/{country_code.lower()}/11_cluster_results/{snapshot}_0-0_1-0_-1_a-infomap_n100_m1-0_s0_c1000.json',country_code,'seqitems', path_prefix='../')
    add_community_to_graph(result)
    sG = sequence_graph(result.graph)
    cG = quotient_graph(sG, 'community', self_loops=False, root_level=None, )
    cG = make_weighted(cG)
    cG.remove_edges_from([(u,v) for u,v,w in cG.edges(data='weight') if w <= weight_threshold])
    internal_references_for_community(result.graph, cG)
    nx.set_node_attributes(cG, nx.get_node_attributes(cG, 'chars_n'), name='community_size')
    nx.set_node_attributes(cG, {
       n:
        cG.nodes[n].get('internal_references', 0)/chars_n if chars_n > 0 else 0
        for n, chars_n in nx.get_node_attributes(cG, 'chars_n').items()
    }, name='internal_density')
    cG = nx.subgraph(cG, [n for n, size in cG.nodes(data='community_size', default=0) if size >= size_threshold])
    nx.set_node_attributes(cG, {n:d/size_divisor for n,d in cG.nodes(data='community_size')}, name='node_size')
    return cG


def make_chapter_graph_for_snapshot(snapshot, country_code, weight_threshold, size_threshold, size_divisor):
    G = nx.read_gpickle(f'../../legal-networks-data/{country_code.lower()}/4_crossreference_graph/seqitems/{snapshot}.gpickle.gz')
    # making this quotient graph takes too much time, implementation probably inefficient
    G, _ = quotient_graph_with_merge(G, self_loops=True, merge_threshold=-1)
    # remove all nodes above the chapter level
    G.remove_nodes_from(set(e[-1] for e in G.nodes(data='parent_key')))
    G.remove_node('root')
    G = make_weighted(G)
    nx.set_node_attributes(G, {u:w for u,v,w in G.edges(data='weight') 
                                    if u == v}, name='self_references'
                              )
    if country_code.lower() == 'de':
        node_ids = {
            n: extract_lawname_from_nodekey(n,'de') 
            if data['level'] == 0 
            else f'{extract_lawname_from_nodekey(n, "de")}/{data["heading"].split("Buch")[0]}Buch' 
            for n, data in G.nodes(data=True)
        }
        
    else: # assuming 'us'
        node_ids = {
            # Title number plus chapter/etc. name
            n: data['law_name'].split('-')[0].split(' ')[-1] 
            if data['level'] == 0
            else f"{data['law_name'].split('-', 1)[0].split(' ', 1)[-1]}/{data['heading'].split('-', 1)[0].split(' ', 1)[-1]}".replace("&ndash;", '-')
            for n, data in G.nodes(data=True)
        }
    nx.set_node_attributes(G, node_ids, name='label')
    nx.set_node_attributes(G, {node_id:node_size/size_divisor for node_id, node_size in dict(G.nodes(data='tokens_n', default=0)).items()}, 
                           name='node_size')
    nx.set_node_attributes(G, {law_name:data['self_references']/data['tokens_n'] 
                                if data.get('self_references',0) > 0 else 0.
                                for law_name, data in G.nodes(data=True)
                               }, name='internal_density'
                          )
    G.remove_edges_from([(u,v) for u,v,w in G.edges(data='weight') if w <= weight_threshold])
    G.remove_edges_from([(u,v) for u,v in G.edges() if u == v])
    G = nx.subgraph(G, [n for n, size in G.nodes(data='tokens_n', default=0) if size >= size_threshold])
    return G


def sort_edges_by_weight(G, reverse=False, filter_weight=0):
    edges_sorted = [e for e in sorted(G.edges(data=True), key=lambda tup:tup[-1]['weight'], reverse=reverse) if e[-1]['weight'] >= filter_weight]
    edgeweights_sorted = [e[-1]['weight'] for e in edges_sorted]
    return edges_sorted, edgeweights_sorted


def sort_nodes_by_size(cG, size_attribute, color_attribute):
    nodes_sorted = sorted([(c, data[size_attribute], data[color_attribute]) for c, data in cG.nodes(data=True)],
                          key=lambda tup:tup[1], reverse=True
                         )
    return zip(*nodes_sorted)


def plot_community_graph(cG, size_attribute='node_size', color_attribute='internal_density',
                         labels=None, close=False, figsize=(16,16), edge_filter_weight=0, 
                         savepath=None, size_divisor=None,
                         cmap=cm.Reds
                        ):
    assert size_divisor
    plt.rcParams['figure.figsize'] = figsize
    if labels is None:
        labels = {node_id:node_id for node_id in cG.nodes()}
    np.random.seed(1234)
    pos = nx.fruchterman_reingold_layout(cG, k=2.2) # the seed argument of nx doesn't do its job properly, hence we use numpy
    edges_sorted, edgeweights_sorted = sort_edges_by_weight(cG, reverse=False, filter_weight=edge_filter_weight)
    nodes_sorted, node_sizes_sorted, node_colors_sorted = sort_nodes_by_size(cG, size_attribute, color_attribute)

    min_max_row = dict(
        edge_weight_min= min(edgeweights_sorted),
        edge_weight_max= max(edgeweights_sorted),
        node_size_min= min(node_sizes_sorted)*size_divisor,
        node_size_max= max(node_sizes_sorted)*size_divisor,
        node_color_min= min(node_colors_sorted),
        node_color_max= max(node_colors_sorted)
    )
    print(min_max_row)

    edges = nx.draw_networkx_edges(cG, pos=pos, width=[e/10 for e in edgeweights_sorted],
                           edgelist=edges_sorted,
                           edge_color=edgeweights_sorted, edge_cmap=cm.Greys,
                           edge_vmax=100, edge_vmin=0,arrowstyle='fancy', arrowsize=40, connectionstyle='arc3,rad=0.2',
                           #min_source_margin=0, min_target_margin=0, # margins don't seem to work
                          )
    for i in range(len(edges_sorted)):
        # we can set attrs of edges using the setters of FancyArrowPatch
        # need to do this, inter alia, b/c alpha is not working in the default drawing function
        # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.patches.FancyArrowPatch.html#matplotlib.patches.FancyArrowPatch
        edges[i].set_alpha(0.5)
        edges[i].set_linewidth(0)
        #edges[i].set_zorder(i)
    nodes = nx.draw_networkx_nodes(cG, nodelist=nodes_sorted, pos=pos, node_size=node_sizes_sorted,node_color=node_colors_sorted, 
                                   cmap=cmap, alpha=0.75)
    #for n in sorted([c[0] for c in cG.nodes(data='node_size')], key=lambda c:c[-1], reverse=True):
    #    nx.draw_networkx_nodes(cG, nodelist=[n], pos=pos, node_size=node_sizes[n], node_color=[node_colors[n]], cmap=cm.Reds)
    nx.draw_networkx_labels(cG, pos=pos, labels=labels)
    plt.axis('off')
    plt.tight_layout()
    if close:
        plt.close()
    if savepath is not None:
        plt.savefig(savepath)
    return min_max_row

In [ ]:
def add_cluster_families_to_lawname_graph(qG, cluster_evolution_graph_config, country_code):
    C = nx.read_gpickle(f'../../legal-networks-data/{country_code.lower()}/13_cluster_evolution_graph/all_{cluster_evolution_graph_config}.gpickle.gz')
    cluster_fam_sorted = cluster_families(C, .15)

    family_nodes = {
        n: idx
        for idx, family in enumerate(cluster_fam_sorted)
        for cluster in family
        for n in C.nodes[cluster]['nodes_contained'].split(',')
    }
    nx.set_node_attributes(qG, family_nodes, 'cluster_family')
    
    nx.set_node_attributes(
        qG, 
        {
            n: min(f,20)
            for n, f in family_nodes.items()
        },
        'cluster_family_color'
    )
    
def add_cluster_families_to_community_graph(qG, cluster_evolution_graph_config, country_code, year):
    C = nx.read_gpickle(f'../../legal-networks-data/{country_code.lower()}/13_cluster_evolution_graph/all_{cluster_evolution_graph_config}.gpickle.gz')
    cluster_fam_sorted = cluster_families(C, .15)
    
    family_nodes = {
        int(cluster.split('_')[1]): idx
        for idx, family in enumerate(cluster_fam_sorted)
        for cluster in family
        if cluster.startswith(year)
    }
    nx.set_node_attributes(qG, family_nodes, 'cluster_family')
    
    nx.set_node_attributes(
        qG, 
        {
            n: min(f,20)
            for n, f in family_nodes.items()
        },
        'cluster_family_color'
    )
    
def get_labels(qG, n_label_threshold):
    nodes_to_label = list(map(lambda tup:tup[0], sorted(list(qG.nodes(data='tokens_n')), key=lambda tup:tup[-1], reverse=True)[:n_label_threshold]))
    labels = {q:qG.nodes[q]["label"] if q in nodes_to_label  else "" for q in qG.nodes()}
    return labels

### Quotient graphs by Title and Title/Chapter resp. Law Name and Law Name/Book

In [ ]:
cluster_evolution_graph_config = '0-0_1-0_-1_a-infomap_n100_m1-0_s0_c1000'

In [ ]:
n_label_threshold = 50

#### US

In [ ]:
country_code = 'us'
weight_threshold = 0
edge_filter_weight = 0

In [ ]:
snapshot = '1994'

In [ ]:
size_divisor = 4000
size_threshold = 0
qG = make_lawname_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                     size_threshold=size_threshold, size_divisor=size_divisor)
row = plot_community_graph(qG, edge_filter_weight=edge_filter_weight, size_attribute='node_size', labels=dict(qG.nodes(data='label')),
                    savepath=f'../graphics/lawname-graph-{snapshot}-{country_code.lower()}.pdf', size_divisor=size_divisor
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='lawname', **row), ignore_index=True)

In [ ]:
size_divisor = 100
size_threshold = 5000
qG = make_chapter_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                     size_threshold=size_threshold, size_divisor=size_divisor)
add_cluster_families_to_lawname_graph(qG, cluster_evolution_graph_config, country_code)

In [ ]:
labels = get_labels(qG, n_label_threshold)

In [ ]:
row = plot_community_graph(qG, edge_filter_weight=edge_filter_weight, size_attribute='node_size', labels=labels,
                    savepath=f'../graphics/chapter-graph-{snapshot}-{country_code.lower()}.pdf', size_divisor=size_divisor,
                    color_attribute='cluster_family_color', cmap=cluster_family_plt_colors(country_code)
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='chapter', **row), ignore_index=True)

In [ ]:
snapshot = '2018'

In [ ]:
size_divisor = 4000
size_threshold = 0
qG = make_lawname_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                     size_threshold=size_threshold, size_divisor=size_divisor)
row = plot_community_graph(qG, edge_filter_weight=edge_filter_weight, size_attribute='node_size', labels=dict(qG.nodes(data='label')),
                    savepath=f'../graphics/lawname-graph-{snapshot}-{country_code.lower()}.pdf', size_divisor=size_divisor
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='lawname', **row), ignore_index=True)

In [ ]:
size_divisor = 100
size_threshold = 5000
qG = make_chapter_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                     size_threshold=size_threshold, size_divisor=size_divisor)
add_cluster_families_to_lawname_graph(qG, cluster_evolution_graph_config, country_code)

In [ ]:
labels = get_labels(qG, n_label_threshold)

In [ ]:
row = plot_community_graph(qG, edge_filter_weight=edge_filter_weight, size_attribute='node_size', labels=labels,
                    savepath=f'../graphics/chapter-graph-{snapshot}-{country_code.lower()}.pdf', size_divisor=size_divisor,
                    color_attribute='cluster_family_color', cmap=cluster_family_plt_colors(country_code)
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='chapter', **row), ignore_index=True)

#### DE

In [ ]:
country_code = 'de'
weight_threshold = 0
edge_filter_weight = 0

In [ ]:
snapshot = '1994-01-01'

In [ ]:
size_divisor = 100
size_threshold = 5000
qG = make_lawname_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                     size_threshold=size_threshold, size_divisor=size_divisor)
labels = {x:key.split('_',2)[1] for x, key in qG.nodes(data='key')}
row = plot_community_graph(qG, edge_filter_weight=edge_filter_weight, size_attribute='node_size', labels=labels,
                    savepath=f'../graphics/lawname-graph-{snapshot[:4]}-{country_code.lower()}.pdf', size_divisor=size_divisor
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='lawname', **row), ignore_index=True)

In [ ]:
size_divisor = 100
size_threshold = 5000
qG = make_chapter_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                     size_threshold=size_threshold, size_divisor=size_divisor)
add_cluster_families_to_lawname_graph(qG, cluster_evolution_graph_config, country_code)

In [ ]:
labels = get_labels(qG, n_label_threshold)

In [ ]:
row = plot_community_graph(qG, edge_filter_weight=edge_filter_weight, size_attribute='node_size', labels=labels,
                    savepath=f'../graphics/chapter-graph-{snapshot[:4]}-{country_code.lower()}.pdf', size_divisor=size_divisor,
                    color_attribute='cluster_family_color', cmap=cluster_family_plt_colors(country_code)
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='chapter', **row), ignore_index=True)

In [ ]:
snapshot = '2018-01-01'

In [ ]:
size_divisor = 100
size_threshold = 5000
qG = make_lawname_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                     size_threshold=size_threshold, size_divisor=size_divisor)
labels = {x:key.split('_',2)[1] for x, key in qG.nodes(data='key')}
row = plot_community_graph(qG, edge_filter_weight=edge_filter_weight, size_attribute='node_size', labels=labels,
                    savepath=f'../graphics/lawname-graph-{snapshot[:4]}-{country_code}.pdf', size_divisor=size_divisor
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='lawname', **row), ignore_index=True)

In [ ]:
size_divisor = 100
size_threshold = 5000
qG = make_chapter_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                     size_threshold=size_threshold, size_divisor=size_divisor)
add_cluster_families_to_lawname_graph(qG, cluster_evolution_graph_config, country_code)

In [ ]:
labels = get_labels(qG, n_label_threshold)
for (k,v) in labels.items():
    if ('_BGB_' in k or '_ZPO_' in k) and v:
        labels[k] = f"{labels[k].split('/')[0]}/{' '.join(qG.nodes[k]['heading'].split(' ')[:2])}"

In [ ]:
row = plot_community_graph(qG, edge_filter_weight=edge_filter_weight, size_attribute='node_size', labels=labels,
                    savepath=f'../graphics/chapter-graph-{snapshot[:4]}-{country_code}.pdf', size_divisor=size_divisor,
                    color_attribute='cluster_family_color', cmap=cluster_family_plt_colors(country_code)
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='chapter', **row), ignore_index=True)

### Quotient graphs by community

In [ ]:
edge_filter_weight = 0
weight_threshold = 0
size_divisor = 4000

#### US

In [ ]:
country_code = 'us'

In [ ]:
snapshot = '1994'

In [ ]:
size_threshold = 100000
cG = make_community_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                       size_threshold=size_threshold, size_divisor=size_divisor)
add_cluster_families_to_community_graph(cG, cluster_evolution_graph_config, country_code, snapshot)
row = plot_community_graph(cG, size_attribute='node_size', edge_filter_weight=edge_filter_weight,
                    savepath=f'../graphics/community-graph-{snapshot}-{country_code}.pdf', size_divisor=size_divisor,
                    color_attribute='cluster_family_color', cmap=cluster_family_plt_colors(country_code)
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='community', **row), ignore_index=True)

In [ ]:
snapshot = '2018'

In [ ]:
size_threshold = 150000
cG = make_community_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                       size_threshold=size_threshold, size_divisor=size_divisor)
add_cluster_families_to_community_graph(cG, cluster_evolution_graph_config, country_code, snapshot)
row = plot_community_graph(cG, size_attribute='node_size', edge_filter_weight=edge_filter_weight,
                     savepath=f'../graphics/community-graph-{snapshot}-{country_code}.pdf', size_divisor=size_divisor,
                    color_attribute='cluster_family_color', cmap=cluster_family_plt_colors(country_code)
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='community', **row), ignore_index=True)

#### DE

In [ ]:
country_code = 'de'

In [ ]:
snapshot = '1994-01-01'

In [ ]:
size_threshold = 100000
cG = make_community_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                       size_threshold=size_threshold, size_divisor=size_divisor)
add_cluster_families_to_community_graph(cG, cluster_evolution_graph_config, country_code, snapshot)
row = plot_community_graph(cG, size_attribute='node_size', edge_filter_weight=edge_filter_weight,
                    savepath=f'../graphics/community-graph-{snapshot[:4]}-{country_code}.pdf', size_divisor=size_divisor,
                    color_attribute='cluster_family_color', cmap=cluster_family_plt_colors(country_code)
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='community', **row), ignore_index=True)

In [ ]:
snapshot = '2018-01-01'

In [ ]:
size_threshold = 150000
cG = make_community_graph_for_snapshot(snapshot, country_code, weight_threshold=weight_threshold, 
                                       size_threshold=size_threshold, size_divisor=size_divisor)
add_cluster_families_to_community_graph(cG, cluster_evolution_graph_config, country_code, snapshot)
row = plot_community_graph(cG, size_attribute='node_size', edge_filter_weight=edge_filter_weight,
                     savepath=f'../graphics/community-graph-{snapshot[:4]}-{country_code}.pdf', size_divisor=size_divisor,
                    color_attribute='cluster_family_color', cmap=cluster_family_plt_colors(country_code)
                    )
min_max_df = min_max_df.append(dict(country_code=country_code, snapshot=snapshot, graph_type='community', **row), ignore_index=True)

### Min-max-table output

In [ ]:
min_max_df.to_csv(f'../graphics/graph-min-max.csv', index=False)

In [ ]:
min_max_df = pd.read_csv(f'../graphics/graph-min-max.csv')
min_max_df.graph_type = [g.capitalize() for g in min_max_df.graph_type]
min_max_df.snapshot = [s[:4] for s in min_max_df.snapshot]
min_max_df.country_code = [c.upper() for c in min_max_df.country_code]
min_max_df.edge_weight_max = min_max_df.edge_weight_max.astype('int64')
min_max_df.edge_weight_min = min_max_df.edge_weight_min.astype('int64')
min_max_df.node_size_max = min_max_df.node_size_max.astype('int64')
min_max_df.node_size_min = min_max_df.node_size_min.astype('int64')

In [ ]:
sorted_columns = sorted(min_max_df.columns, key=lambda c: {
    'graph_type': 'a',
    'country_code': 'b',
    'snapshot': 'c',
}.get(c, 'x') + c.replace(' min', ' amin'))
min_max_df = min_max_df[sorted_columns]
min_max_df = min_max_df[min_max_df.graph_type != 'Lawname']
min_max_df.columns = [c+'.' if c.endswith('max') or c.endswith('min') else c for c in min_max_df.columns] 
min_max_df.columns = ['\multicolumn{1}{p{20mm}}{\centering ' + c.replace('_', ' ').capitalize().replace(' ', '\\\\') + '}' for c in min_max_df.columns]

In [ ]:
with open(f'../graphics/graph-min-max-table.tex', 'w') as f:
    min_max_df[[c for c in min_max_df.columns if 'color' not in c]].to_latex(f, escape=False, index=False)

### End